In [1]:
import pandas as pd
data1 = "1_paid_marketing.csv"
data2 = "2_hcp_data.csv"
data3 = "3_shifts_data.csv"
campaigns = pd.read_csv(data1)
hcp = pd.read_csv(data2)
shifts = pd.read_csv(data3)

In [2]:
campaigns.head()

,CAMPAIGN_ID,CAMPAIGN_TYPE,DAY,COST,IMPRESSIONS,CLICKS
0,13743509691,Google Search,9/1/22,$199.36,"1,184",57
1,13743509691,Google Search,9/3/22,$108.69,105,31
2,13743509691,Google Search,9/4/22,$188.79,794,49
3,13743509691,Google Search,9/5/22,$198.47,"1,069",52
4,13743509691,Google Search,9/8/22,$900.51,"2,979",213


In [3]:
campaigns.columns

Index(['CAMPAIGN_ID', 'CAMPAIGN_TYPE', 'DAY', ' COST ', ' IMPRESSIONS ',
       ' CLICKS '],
      dtype='object')

In [4]:
# Remove unnecessary columns
del campaigns['CAMPAIGN_TYPE']
del campaigns[' IMPRESSIONS ']
del campaigns[' CLICKS ']


# Rename column
campaigns = campaigns.rename(columns={' COST ':'COST'})

In [5]:
campaigns.count()

CAMPAIGN_ID    3202
DAY            3202
COST           3202
dtype: int64

In [6]:
campaigns.isnull().sum()

CAMPAIGN_ID    0
DAY            0
COST           0
dtype: int64

In [7]:
# Check datatype of each column
campaigns.dtypes

CAMPAIGN_ID    object
DAY            object
COST           object
dtype: object

In [8]:
# Check if data can be converted to new datatype
print(pd.to_datetime(campaigns['DAY'], errors='coerce').isnull().value_counts())
print(pd.to_numeric(campaigns['COST'], errors='coerce').isnull().value_counts())

False    3202
Name: DAY, dtype: int64
True    3202
Name: COST, dtype: int64


In [9]:
# If errors above, check for cause
print(pd.to_numeric(campaigns['COST'], errors='raise'))

ValueError: Unable to parse string " $199.36 " at position 0

In [10]:
# Fix errors
campaigns['COST'] = campaigns['COST'].str.replace('$','')
campaigns['COST'] = campaigns['COST'].str.replace(',','')

In [11]:
# Confirm errors are resolved
print(pd.to_datetime(campaigns['DAY'], errors='coerce').isnull().value_counts())
print(pd.to_numeric(campaigns['COST'], errors='coerce').isnull().value_counts())

False    3202
Name: DAY, dtype: int64
False    3202
Name: COST, dtype: int64


In [12]:
# Confirm datatype conversion
campaigns.DAY = pd.to_datetime(campaigns['DAY'], errors='coerce')
campaigns.COST = pd.to_numeric(campaigns['COST'], errors='coerce')

print(campaigns.isnull().sum())
print(campaigns.shape)
print(campaigns.dtypes)

CAMPAIGN_ID    0
DAY            0
COST           0
dtype: int64
(3202, 3)
CAMPAIGN_ID            object
DAY            datetime64[ns]
COST                  float64
dtype: object


In [13]:
campaigns.head()

,CAMPAIGN_ID,DAY,COST
0,13743509691,2022-09-01,199.36
1,13743509691,2022-09-03,108.69
2,13743509691,2022-09-04,188.79
3,13743509691,2022-09-05,198.47
4,13743509691,2022-09-08,900.51


In [14]:
# Reset index to date and filter only October dates to new df
campaigns = campaigns.set_index('DAY')
oct_campaigns = campaigns.loc['2022-10']
print(oct_campaigns.count())

CAMPAIGN_ID    1260
COST           1260
dtype: int64


In [15]:
oct_campaigns.head()

,CAMPAIGN_ID,COST
DAY,,
2022-10-01,13743509691,309.91
2022-10-02,13743509691,320.63
2022-10-03,13743509691,446.41
2022-10-06,13743509691,335.31
2022-10-08,13743509691,277.07


In [16]:
# Calculate total spent in October
oct_campaign_total = oct_campaigns['COST'].sum()
oct_campaign_total

815061.3

In [17]:
hcp.head()

,HCP_ID,MSA,QUALIFICATION,CREATED_AT,LICENSE_REVIEWED,ONBOARD_AT,FIRST_CLAIM_TIME,FIRST_SHIFT_TIME,LIFETIME_SHIFTS,REFERRER,CAMPAIGN_KEY
0,633be91bfb9a096b9e4d3a82,"Wichita, KS",CNA,10/5/22,10/13/22,10/29/22,11/1/22,11/21/22,2,NaN,17631861264
1,632d018f9603d7808339a6bb,"Tampa-St. Petersburg-Clearwater, FL",CNA,9/23/22,NaN,NaN,NaN,NaN,,NaN,17631861264
2,6361931a7ccb0c3b06407c33,"Scranton--Wilkes-Barre, PA",RN,11/1/22,11/1/22,NaN,NaN,NaN,,NaN,18530777721
3,62feb63c88ad3001baf3108f,"St. Louis, MO-IL",CNA,8/18/22,8/18/22,NaN,NaN,NaN,,NaN,17631861264
4,63841ac43540dfe4c2ec0996,"Cincinnati, OH-KY-IN",CNA,11/28/22,11/28/22,NaN,NaN,NaN,,NaN,18530777721


In [18]:
# Check columns and datatypes
hcp.columns

Index(['HCP_ID', 'MSA', 'QUALIFICATION', 'CREATED_AT', 'LICENSE_REVIEWED',
       'ONBOARD_AT', 'FIRST_CLAIM_TIME', 'FIRST_SHIFT_TIME', 'LIFETIME_SHIFTS',
       'REFERRER', 'CAMPAIGN_KEY'],
      dtype='object')

In [19]:
hcp.count()

HCP_ID              67317
MSA                 67317
QUALIFICATION       67317
CREATED_AT          67317
LICENSE_REVIEWED    60253
ONBOARD_AT          26805
FIRST_CLAIM_TIME    17950
FIRST_SHIFT_TIME    16350
LIFETIME_SHIFTS     67317
REFERRER             9167
CAMPAIGN_KEY        67317
dtype: int64

In [20]:
print(hcp.isnull().sum())

HCP_ID                  0
MSA                     0
QUALIFICATION           0
CREATED_AT              0
LICENSE_REVIEWED     7064
ONBOARD_AT          40512
FIRST_CLAIM_TIME    49367
FIRST_SHIFT_TIME    50967
LIFETIME_SHIFTS         0
REFERRER            58150
CAMPAIGN_KEY            0
dtype: int64


In [21]:
# Remove all null values in first_shift_time, ie. remove registrants who never worked
hcp.dropna(subset = ['FIRST_SHIFT_TIME'], inplace=True)
print(hcp.isnull().sum())

HCP_ID                 0
MSA                    0
QUALIFICATION          0
CREATED_AT             0
LICENSE_REVIEWED      38
ONBOARD_AT           376
FIRST_CLAIM_TIME       0
FIRST_SHIFT_TIME       0
LIFETIME_SHIFTS        0
REFERRER            9809
CAMPAIGN_KEY           0
dtype: int64


In [22]:
hcp.dtypes

HCP_ID              object
MSA                 object
QUALIFICATION       object
CREATED_AT          object
LICENSE_REVIEWED    object
ONBOARD_AT          object
FIRST_CLAIM_TIME    object
FIRST_SHIFT_TIME    object
LIFETIME_SHIFTS     object
REFERRER            object
CAMPAIGN_KEY        object
dtype: object

In [23]:
print(pd.to_numeric(hcp['LIFETIME_SHIFTS'], errors='raise'))

0         2
7        23
16       49
23       13
31        2
36       33
37       22
44        6
45       20
51        8
60        8
61       70
74        2
80       21
81       10
86       17
92       25
93        4
94       11
95       21
97       17
105      23
107       6
118       6
121      13
122      24
130       2
140      10
143      20
154      20
         ..
67189    13
67190    27
67193    17
67198     2
67203    10
67207    25
67211    10
67213     4
67221     4
67228     6
67232    44
67240    20
67244    20
67248    22
67253     2
67257     2
67264    15
67272     6
67277    28
67278     8
67284    47
67292    30
67302    23
67305    11
67306    24
67308    20
67310    10
67311     4
67313    10
67315    19
Name: LIFETIME_SHIFTS, Length: 16350, dtype: int64


In [24]:
hcp.CREATED_AT = pd.to_datetime(hcp['CREATED_AT'], errors='coerce')
hcp.LICENSE_REVIEWED = pd.to_datetime(hcp['LICENSE_REVIEWED'], errors='coerce')
hcp.ONBOARD_AT = pd.to_datetime(hcp['ONBOARD_AT'], errors='coerce')
hcp.FIRST_CLAIM_TIME = pd.to_datetime(hcp['FIRST_CLAIM_TIME'], errors='coerce')
hcp.FIRST_SHIFT_TIME = pd.to_datetime(hcp['FIRST_SHIFT_TIME'], errors='coerce')
hcp.LIFETIME_SHIFTS = pd.to_numeric(hcp['LIFETIME_SHIFTS'], errors='coerce')

print(hcp.isnull().sum())
print(hcp.shape)
print(hcp.dtypes)

HCP_ID                 0
MSA                    0
QUALIFICATION          0
CREATED_AT             0
LICENSE_REVIEWED      38
ONBOARD_AT           376
FIRST_CLAIM_TIME       0
FIRST_SHIFT_TIME       0
LIFETIME_SHIFTS        0
REFERRER            9809
CAMPAIGN_KEY           0
dtype: int64
(16350, 11)
HCP_ID                      object
MSA                         object
QUALIFICATION               object
CREATED_AT          datetime64[ns]
LICENSE_REVIEWED    datetime64[ns]
ONBOARD_AT          datetime64[ns]
FIRST_CLAIM_TIME    datetime64[ns]
FIRST_SHIFT_TIME    datetime64[ns]
LIFETIME_SHIFTS              int64
REFERRER                    object
CAMPAIGN_KEY                object
dtype: object


In [25]:
# Filter only HCP that were acquired in Oct
hcp = hcp.set_index('CREATED_AT')
oct_hcp = hcp.loc['2022-10']
print(oct_hcp.count())

HCP_ID              1340
MSA                 1340
QUALIFICATION       1340
LICENSE_REVIEWED    1335
ONBOARD_AT          1330
FIRST_CLAIM_TIME    1340
FIRST_SHIFT_TIME    1340
LIFETIME_SHIFTS     1340
REFERRER             650
CAMPAIGN_KEY        1340
dtype: int64


In [26]:
# Sum of October HCP acquisitions
oct_hcp_acq = oct_hcp['HCP_ID'].count()
oct_hcp_acq

1340

In [27]:
# Calculate referral costs claimed
oct_referral_cost = oct_hcp['REFERRER'].count() * 300
oct_referral_cost

195000

In [28]:
# Calculate CAC: (click ads + referral fees) / HCP acquired in October
CAC = (oct_campaign_total + oct_referral_cost) / oct_hcp_acq
CAC

753.7770895522389